In [1]:
import pandas as pd
import sqlite3
import numpy as np
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [2]:
#db = sqlite3.connect('data/hop_team.sqlite')
#db.execute('CREATE INDEX from_npi ON calls(from_npi)')
#db.close()

In [3]:
query = '''WITH tax_nucc AS(
            SELECT * 
            FROM taxonomy as t
            JOIN nucc as n
            ON t.taxonomy_code=n.code
            WHERE taxonomy_switch = 'Y' 
            ),
            trim_npi AS(
            SELECT npi, entity_type_code, `provider_organization_name_(legal_business_name)`,`provider_last_name_(legal_name)`,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_other_organization_name,provider_first_line_business_mailing_address,provider_second_line_business_mailing_address,provider_business_mailing_address_city_name,provider_business_mailing_address_state_name,provider_business_mailing_address_postal_code,
                CASE WHEN `provider_organization_name_(legal_business_name)` LIKE '%Vanderbilt%' THEN 'Vanderbilt'
                ELSE '-' END AS facility_consolidation
            FROM npi
            WHERE provider_business_mailing_address_state_name = 'TN'
            ),
            best_npi AS(
                        SELECT*
                        FROM trim_npi
                        JOIN tax_nucc
                        on trim_npi.npi= tax_nucc.NPI
                        )
            SELECT *
            FROM calls
            JOIN best_npi
            on calls.from_npi=best_npi.npi
            JOIN best_npi AS b1
            on calls.to_npi=b1.npi
            WHERE transaction_count>49
            AND average_day_wait<100
            AND average_day_wait>0
            '''
with sqlite3.connect('data/hop_team.sqlite') as db:
    full = pd.read_sql(query, db)
    
    

In [4]:
full.shape

(852201, 60)

In [5]:
full.info

<bound method DataFrame.info of           from_npi      to_npi  patient_count  transaction_count  \
0       1841293891  1245290857            176                231   
1       1841293891  1245437060             44                 50   
2       1841293891  1306061601             67                 76   
3       1841293891  1356633564             56                 66   
4       1841293891  1457302010            233                366   
...            ...         ...            ...                ...   
852196  1275001851  1184610859             62                 62   
852197  1275001851  1275531956            130                130   
852198  1275001851  1366440570             74                 74   
852199  1275001851  1710919980            108                108   
852200  1275001851  1972648319             84                 84   

        average_day_wait  std_day_wait         npi  entity_type_code  \
0                 28.857        60.036  1841293891               1.0   
1      

In [6]:
list(full)

['from_npi',
 'to_npi',
 'patient_count',
 'transaction_count',
 'average_day_wait',
 'std_day_wait',
 'npi',
 'entity_type_code',
 'provider_organization_name_(legal_business_name)',
 'provider_last_name_(legal_name)',
 'provider_first_name',
 'provider_middle_name',
 'provider_name_prefix_text',
 'provider_name_suffix_text',
 'provider_credential_text',
 'provider_other_organization_name',
 'provider_first_line_business_mailing_address',
 'provider_second_line_business_mailing_address',
 'provider_business_mailing_address_city_name',
 'provider_business_mailing_address_state_name',
 'provider_business_mailing_address_postal_code',
 'facility_consolidation',
 'NPI:1',
 'taxonomy_switch',
 'taxonomy_code',
 'code',
 'grouping',
 'classification',
 'specialization',
 'definition',
 'notes',
 'display_name',
 'section',
 'npi',
 'entity_type_code',
 'provider_organization_name_(legal_business_name)',
 'provider_last_name_(legal_name)',
 'provider_first_name',
 'provider_middle_name',
 'p

In [7]:
last_col= full.pop('to_npi')

In [8]:
full.insert(loc=31, column= 'to_npi', value= last_col)

In [9]:
full.pop('npi')

,npi,npi
0,1841293891,1245290857
1,1841293891,1245437060
2,1841293891,1306061601
3,1841293891,1356633564
4,1841293891,1457302010
...,...,...
852196,1275001851,1184610859
852197,1275001851,1275531956
852198,1275001851,1366440570
852199,1275001851,1710919980


In [10]:
full.pop('NPI:1')

,NPI:1,NPI:1
0,1841293891,1245290857
1,1841293891,1245437060
2,1841293891,1306061601
3,1841293891,1356633564
4,1841293891,1457302010
...,...,...
852196,1275001851,1184610859
852197,1275001851,1275531956
852198,1275001851,1366440570
852199,1275001851,1710919980


In [11]:
full.nunique()

from_npi                                            25307
patient_count                                        4433
transaction_count                                    5858
average_day_wait                                    96768
std_day_wait                                        91994
entity_type_code                                        2
provider_organization_name_(legal_business_name)     4613
provider_last_name_(legal_name)                      9294
provider_first_name                                  3859
provider_middle_name                                 2711
provider_name_prefix_text                               6
provider_name_suffix_text                               6
provider_credential_text                              719
provider_other_organization_name                     2625
provider_first_line_business_mailing_address         7915
provider_second_line_business_mailing_address        1593
provider_business_mailing_address_city_name           348
provider_busin

In [17]:
full.columns=['from_npi',
 'patient_count',
 'transaction_count',
 'average_day_wait',
 'std_day_wait',
 'entity_type_code_1',
 'provider_organization_name_(legal_business_name)_1',
 'provider_last_name_(legal_name)_1',
 'provider_first_name_1',
 'provider_middle_name_1',
 'provider_name_prefix_text_1',
 'provider_name_suffix_text_1',
 'provider_credential_text_1',
 'provider_other_organization_name_1',
 'provider_first_line_business_mailing_address_1',
 'provider_second_line_business_mailing_address_1',
 'provider_business_mailing_address_city_name_1',
 'provider_business_mailing_address_state_name_1',
 'provider_business_mailing_address_postal_code_1',
'facility_consolidation_1',
 'taxonomy_switch_1',
 'taxonomy_code_1',
 'code_1',
 'grouping_1',
 'classification_1',
 'specialization_1',
 'definition_1',
 'notes_1',
 'display_name_1',
 'section_1',
 'to_npi',
 'entity_type_code_2',
 'provider_organization_name_(legal_business_name)_2',
 'provider_last_name_(legal_name)_2',
 'provider_first_name_2',
 'provider_middle_name_2',
 'provider_name_prefix_text_2',
 'provider_name_suffix_text_2',
 'provider_credential_text_2',
 'provider_other_organization_name_2',
 'provider_first_line_business_mailing_address_2',
 'provider_second_line_business_mailing_address_2',
 'provider_business_mailing_address_city_name_2',
 'provider_business_mailing_address_state_name_2',
 'provider_business_mailing_address_postal_code_2',
 'facility_consolidation_2',
 'taxonomy_switch_2',
 'taxonomy_code_2',
 'code_2',
 'grouping_2',
 'classification_2',
 'specialization_2',
 'definition_2',
 'notes_2',
 'display_name_2',
 'section_2']

In [18]:
full

,from_npi,patient_count,transaction_count,average_day_wait,std_day_wait,entity_type_code_1,provider_organization_name_(legal_business_name)_1,provider_last_name_(legal_name)_1,provider_first_name_1,provider_middle_name_1,provider_name_prefix_text_1,provider_name_suffix_text_1,provider_credential_text_1,provider_other_organization_name_1,provider_first_line_business_mailing_address_1,provider_second_line_business_mailing_address_1,provider_business_mailing_address_city_name_1,provider_business_mailing_address_state_name_1,provider_business_mailing_address_postal_code_1,facility_consolidation_1,taxonomy_switch_1,taxonomy_code_1,code_1,grouping_1,classification_1,specialization_1,definition_1,notes_1,display_name_1,section_1,to_npi,entity_type_code_2,provider_organization_name_(legal_business_name)_2,provider_last_name_(legal_name)_2,provider_first_name_2,provider_middle_name_2,provider_name_prefix_text_2,provider_name_suffix_text_2,provider_credential_text_2,provider_other_organization_name_2,provider_first_line_business_mailing_address_2,provider_second_line_business_mailing_address_2,provider_business_mailing_address_city_name_2,provider_business_mailing_address_state_name_2,provider_business_mailing_address_postal_code_2,facility_consolidation_2,taxonomy_switch_2,taxonomy_code_2,code_2,grouping_2,classification_2,specialization_2,definition_2,notes_2,display_name_2,section_2
0,1841293891,176,231,28.857,60.036,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,-,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,1245290857,Individual,2.0,RADIOLOGY IMAGING ASSOCIATES OF OAK RIDGE PC,None,None,None,None,None,None,RADIOLOGY IMAGING ASSOCIATES OF OAK RIDGE,601 DODDS AVE,None,CHATTANOOGA,TN,374043911.0,-,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,Individual
1,1841293891,44,50,35.260,60.604,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,-,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,1245437060,Individual,1.0,None,MONAZAM TAVALLAEE,MEHRAN,None,None,None,M.D.,None,49 CLEVELAND ST,SUITE 350,CROSSVILLE,TN,385559716.0,-,Y,207R00000X,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,None,"A physician who provides long-term, comprehens...","Source: American Board of Medical Specialties,...",Internal Medicine Physician,Individual
2,1841293891,67,76,53.184,83.149,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,-,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,1306061601,Individual,1.0,None,LEMOND,THOMAS,CHARLES,DR.,None,M.D.,None,990 OAK RIDGE TPKE,None,OAK RIDGE,TN,378306976.0,-,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,Individual
3,1841293891,56,66,61.894,81.764,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,-,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,1356633564,Individual,1.0,None,LUTTRELL,LINDSAY,GEORGE,DR.,None,M.D.,None,2508 WILD FERN LN,None,KNOXVILLE,TN,379313359.0,-,Y,2085R0202X,2085R0202

In [19]:
full_2=full[full['entity_type_code_1']==1.0]

In [20]:
full_2=full_2[full_2['entity_type_code_2']==2.0]

In [64]:
full_2

,from_npi,patient_count,transaction_count,average_day_wait,std_day_wait,entity_type_code_1,provider_organization_name_(legal_business_name)_1,provider_last_name_(legal_name)_1,provider_first_name_1,provider_middle_name_1,provider_name_prefix_text_1,provider_name_suffix_text_1,provider_credential_text_1,provider_other_organization_name_1,provider_first_line_business_mailing_address_1,provider_second_line_business_mailing_address_1,provider_business_mailing_address_city_name_1,provider_business_mailing_address_state_name_1,provider_business_mailing_address_postal_code_1,facility_consolidation_1,taxonomy_switch_1,taxonomy_code_1,code_1,grouping_1,classification_1,specialization_1,definition_1,notes_1,display_name_1,section_1,to_npi,entity_type_code_2,provider_organization_name_(legal_business_name)_2,provider_last_name_(legal_name)_2,provider_first_name_2,provider_middle_name_2,provider_name_prefix_text_2,provider_name_suffix_text_2,provider_credential_text_2,provider_other_organization_name_2,provider_first_line_business_mailing_address_2,provider_second_line_business_mailing_address_2,provider_business_mailing_address_city_name_2,provider_business_mailing_address_state_name_2,provider_business_mailing_address_postal_code_2,facility_consolidation_2,taxonomy_switch_2,taxonomy_code_2,code_2,grouping_2,classification_2,specialization_2,definition_2,notes_2,display_name_2,section_2,full_addy_1,full_addy_2,from_npi_pt_count,from_npi_transaction_count,to_npi_pt_count,to_npi_transaction_count,from_facility_pt_count,from_facility_transaction_count,to_facility_pt_count,to_facility_transaction_count
0,1841293891,176,231,28.857,60.036,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,-,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,1245290857,Individual,2.0,RADIOLOGY IMAGING ASSOCIATES OF OAK RIDGE PC,None,None,None,None,None,None,RADIOLOGY IMAGING ASSOCIATES OF OAK RIDGE,601 DODDS AVE,None,CHATTANOOGA,TN,374043911.0,-,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,Individual,"49 CLEVELAND ST 310, CROSSVILLE","601 DODDS AVE, CHATTANOOGA",766,1130,36666932,52211903,766,1130,385028,489261
4,1841293891,233,366,3.101,19.513,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,-,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,1457302010,Individual,2.0,"COVENANT MEDICAL GROUP, INC.",None,None,None,None,None,None,None,1400 CENTERPOINT BLVD,"BLDG A, SUITE 202",KNOXVILLE,TN,379322146.0,-,Y,207Q00000X,207Q00000X,Allopathic & Osteopathic Physicians,Family Medicine,None,Family Medicine is the medical specialty which...,Source: American Board of Family Medicine [1/...,Family Medicine Physician,Individual,"49 CLEVELAND ST 310, CROSSVILLE","1400 CENTERPOINT BLVD, KNOXVILLE",766,1130,36666932,52211903,766,1130,322519,477249
6,1841293891,63,80,32.675,59.438,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,-,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,1518980499,Individual,2.0,"BUCKEYE HOME HEALTH CENTER, INC.",None,None,None,None,None,None,None,PO BOX 1197,None,JAMESTOWN,TN,38556.0,-,Y,332B00000X,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,None,A supplier of medical equipment such as respir...,None,Durable Medical Equipment & Medical Supplies,Non-Individual,"49 CLEVELAND ST 310, CROSSVILL

In [22]:
# 1.  using full_2 determine who is sending the most pts and initiating the most transactions
### solve: sum of patient_count grouped by from_npi and sum of transaction_count  grouped by from_npi
###### visualization: could have a bar graph with all NPIs, but that will be too much information, maybe a stripped down with
#####                 most, least and median NPI and most, least and median facility overall, maybe  a histogram to show trends
#####                 super_extra_credit: r_shiny app that allows you to use drop down to visualize by NPI or facility,and
#####                 filter by specialty, and uses specialty to look at wait-time.  Further points if there can be a slide 
#####                 with the Neo4J stuff, and one more that        

# 2.  using full_2 determine who is receiving the most pts and the most transactions
### solve: sum of patient_count grouped by to_npi and sum of transaction_count  grouped by to_npi
###### visualization: see above


# 3.  use this information to determine which facility gets and receives the most pts and transactions
### solve: group previous info by facility??? make sure that you have consolidated the facilites for accuracy.  May try a string
###        like "%rough_hospital address%" query in sql and add a "renamed_facility_address" column.
###        ~~~on second thought, this would require doing that for every facility, might need fuzzy naming ~~~

# 4.  ***The purpose of the next questions to to see where Vandy is doing well and where they could improve***
#     Use this information to determine which doctors refer to Vanderbilt most (top 5, and top 5 not assocaited with Vanderbilt), 
#     which refer to Vanderbilt least (again, top 5, and top 5 not assocaited with Vanderbilt) as well as (4b) the facilities and 
#     specialites that refer to Vandy the most and least. 

# 5.  Determine doctor's patient caseload as a metric for growth (ei: look at doctors, grouped by speciality to determine which
#     ones have the most and least pts~~~(may not actually work because these are referrals, not total pts)~~~. Use that number 
#     to find out how what a facilities pt.per.physician ratio is in total and per speciality.  This could theorectically also 
#     be a shiny app.)


In [23]:
# (1)group by from_npi summing the patient_count and then sorting the values
pt_count_by_from_npi= full_2.groupby(['from_npi'])['patient_count'].sum().reset_index()
pt_count_by_from_npi. sort_values('patient_count', ascending=False)
#these are the npi's with the 5 most and least pt count

,from_npi,patient_count
1725,1093753303,646023
13238,1710915061,117275
12660,1689673048,87477
11257,1609848563,75274
162,1003921404,70078
...,...,...
521,1023503612,12
4617,1255309381,12
6474,1356319198,12
17818,1962411934,11


In [24]:
# (1) group by from_npi summing the transaction_count and then sorting the values
transaction_count_by_from_npi= full_2.groupby(['from_npi'])['transaction_count'].sum().reset_index()
transaction_count_by_from_npi. sort_values('transaction_count', ascending=False)
#these are the npi's with the 5 most and least transaction count

,from_npi,transaction_count
1725,1093753303,978213
13238,1710915061,156297
12660,1689673048,118935
2841,1154356764,82001
6912,1376658781,79237
...,...,...
15971,1861431884,50
5966,1326054446,50
10648,1578566519,50
8920,1487189197,50


In [25]:
# (2) group by to_npi summing the patient_count and then sorting the values
pt_count_by_to_npi= full_2.groupby(['to_npi'])['patient_count'].sum().reset_index()
pt_count_by_to_npi. sort_values('patient_count', ascending=False)
#these are the npi's with the 5 most and least pt count

,to_npi,patient_count
0,Individual,36666932
1,Non-Individual,1459


In [26]:
# (2) group by to_npi summing the transaction_count and then sorting the values
transaction_count_by_to_npi= full_2.groupby(['to_npi'])['transaction_count'].sum().reset_index()
transaction_count_by_to_npi. sort_values('transaction_count', ascending=False)
#these are the npi's with the 5 most and least transaction count

,to_npi,transaction_count
0,Individual,52211903
1,Non-Individual,1654


In [27]:
full_2['full_addy_1']= full_2.provider_first_line_business_mailing_address_1.map(str) +', ' + full_2.provider_business_mailing_address_city_name_1

In [28]:
full_2['full_addy_2']= full_2.provider_first_line_business_mailing_address_2.map(str) +', ' + full_2.provider_business_mailing_address_city_name_2

In [29]:
full_2['from_npi_pt_count']= full_2.groupby(['from_npi'])['patient_count'].transform(np.sum)

In [30]:
full_2['from_npi_transaction_count']= full_2.groupby(['from_npi'])['transaction_count'].transform(np.sum)

In [31]:
full_2['to_npi_pt_count']= full_2.groupby(['to_npi'])['patient_count'].transform(np.sum)

In [34]:
full_2['to_npi_transaction_count']= full_2.groupby(['to_npi'])['transaction_count'].transform(np.sum)

In [35]:
full_2['provider_first_line_business_mailing_address_1'].nunique()

6083

In [36]:
unique_addy=pd.DataFrame((full_2['provider_first_line_business_mailing_address_1'].unique()))

In [37]:
unique_addy

,0
0,49 CLEVELAND ST 310
1,PO BOX 9
2,PO BOX 440100
3,PO BOX 778
4,127 CRESTVIEW PARK DR
...,...
6078,4405 WASHINGTON CT
6079,PO BOX 750
6080,147 NORTH STAR DRIVE
6081,700 W FOREST AVE STE 200


In [38]:
full_2[full_2['provider_first_line_business_mailing_address_1']=='PO BOX 9']

,from_npi,patient_count,transaction_count,average_day_wait,std_day_wait,entity_type_code_1,provider_organization_name_(legal_business_name)_1,provider_last_name_(legal_name)_1,provider_first_name_1,provider_middle_name_1,provider_name_prefix_text_1,provider_name_suffix_text_1,provider_credential_text_1,provider_other_organization_name_1,provider_first_line_business_mailing_address_1,provider_second_line_business_mailing_address_1,provider_business_mailing_address_city_name_1,provider_business_mailing_address_state_name_1,provider_business_mailing_address_postal_code_1,facility_consolidation_1,taxonomy_switch_1,taxonomy_code_1,code_1,grouping_1,classification_1,specialization_1,definition_1,notes_1,display_name_1,section_1,to_npi,entity_type_code_2,provider_organization_name_(legal_business_name)_2,provider_last_name_(legal_name)_2,provider_first_name_2,provider_middle_name_2,provider_name_prefix_text_2,provider_name_suffix_text_2,provider_credential_text_2,provider_other_organization_name_2,provider_first_line_business_mailing_address_2,provider_second_line_business_mailing_address_2,provider_business_mailing_address_city_name_2,provider_business_mailing_address_state_name_2,provider_business_mailing_address_postal_code_2,facility_consolidation_2,taxonomy_switch_2,taxonomy_code_2,code_2,grouping_2,classification_2,specialization_2,definition_2,notes_2,display_name_2,section_2,full_addy_1,full_addy_2,from_npi_pt_count,from_npi_transaction_count,to_npi_pt_count,to_npi_transaction_count
15,1659374601,60,65,21.646,29.540,1.0,None,OBERDICK,WENDY,TIPTON,None,None,MD,None,PO BOX 9,None,KINGSPORT,TN,376620009.0,-,Y,207Q00000X,207Q00000X,Allopathic & Osteopathic Physicians,Family Medicine,None,Family Medicine is the medical specialty which...,Source: American Board of Family Medicine [1/...,Family Medicine Physician,1841638277,Individual,2.0,BTDI JV LLP,None,None,None,None,None,None,TOUCHSTONE MEDICAL IMAGING AT RED OAK,1431 PERRONE WAY,None,FRANKLIN,TN,370694243.0,-,Y,293D00000X,293D00000X,Laboratories,Physiological Laboratory,None,A laboratory that operates independently of a ...,Source: Paraphrased from the Medicare Carrier ...,Physiological Laboratory,Non-Individual,"PO BOX 9, KINGSPORT","1431 PERRONE WAY, FRANKLIN",60,65,36666932,52211903
694,1255334298,48,52,89.788,88.380,1.0,None,HUNT,JEFFREY,None,DR.,None,MD,None,PO BOX 9,None,KINGSPORT,TN,376620009.0,-,Y,207Y00000X,207Y00000X,Allopathic & Osteopathic Physicians,Otolaryngology,None,An otolaryngologist-head and neck surgeon prov...,"Source: American Board of Medical Specialties,...",Otolaryngology Physician,1003904095,Individual,2.0,WPS PROVIDERS INC,None,None,None,None,None,None,None,105 W STONE DR,SUITE 6A,KINGSPORT,TN,376603365.0,-,Y,208D00000X,208D00000X,Allopathic & Osteopathic Physicians,General Practice,None,Definition to come...,None,General Practice Physician,Individual,"PO BOX 9, KINGSPORT","105 W STONE DR, KINGSPORT",1338,2169,36666932,52211903
695,1255334298,48,51,79.725,78.437,1.0,None,HUNT,JEFFREY,None,DR.,None,MD,None,PO BOX 9,None,KINGSPORT,TN,376620009.0,-,Y,207Y00000X,207Y00000X,Allopathic & Osteopathic Physicians,Otolaryngology,None,An otolaryngologist-head and neck surgeon prov...,"Source: American Board of Medical Specialties,...",Otolaryngology Physician,1093762759,Individual,2.0,MOUNTAIN EMPIRE RADIOLOGY INC PC,None,None,None,None,None,None,None,1301 SUNSET DR,STE 3,JOHNSON CITY,TN,376047906.0,-,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,Individual,"PO BOX 9, KINGSPORT","1301 SUNSET DR, JOHNSON CITY",1338,2169,36666932,52211903
696,1255334298,43,52,64.231,81.937,1.0,None,HUNT,JEFFREY,None,DR.,None,MD,None,PO BOX 9,None,KINGSPORT,TN,376620009.0,-,Y,207Y00000X,207Y00000X,Allopathic & Osteopathic Physicians,Otolaryngology,None,An otolaryngologist-head and neck surgeon prov...,"So

In [39]:
# (3a)group by full_addy_1 summing the patient_count and then sorting the values
pt_count_by_full_addy_1= full_2.groupby(['full_addy_1'])['patient_count'].sum().reset_index()
pt_count_by_full_addy_1. sort_values('patient_count', ascending=False)
#these are the facilities's with the 5 most and least pt count

,full_addy_1,patient_count
1952,"210 25TH AVE N STE 1204, NASHVILLE",991514
3820,"5301 VIRGINIA WAY, BRENTWOOD",923002
2736,"3024 BUSINESS PARK CIR, GOODLETTSVILLE",876060
2365,"2501 CITICO AVE, CHATTANOOGA",549853
5592,"PO BOX 2121, MEMPHIS",505969
...,...,...
2552,"28 MIDWAY STREET, BRISTOL",13
5016,"9051 EXECUTIVE PARK DR STE 500, KNOXVILLE",13
5222,"979 E 3RD ST STE C725, CHATTANOOGA",13
2455,"2611 SALEM CREEK DR, MURFREESBORO",12


In [40]:
full_2['from_facility_pt_count']= full_2.groupby(['full_addy_1'])['patient_count'].transform(np.sum)

In [41]:
full_2['from_facility_transaction_count']= full_2.groupby(['full_addy_1'])['transaction_count'].transform(np.sum)

In [42]:
full_2['to_facility_pt_count']= full_2.groupby(['full_addy_2'])['patient_count'].transform(np.sum)

In [43]:
full_2['to_facility_transaction_count']= full_2.groupby(['full_addy_2'])['transaction_count'].transform(np.sum)

In [44]:
full_2[full_2['full_addy_1']=='210 25TH AVE N STE 1204, NASHVILLE']

,from_npi,patient_count,transaction_count,average_day_wait,std_day_wait,entity_type_code_1,provider_organization_name_(legal_business_name)_1,provider_last_name_(legal_name)_1,provider_first_name_1,provider_middle_name_1,provider_name_prefix_text_1,provider_name_suffix_text_1,provider_credential_text_1,provider_other_organization_name_1,provider_first_line_business_mailing_address_1,provider_second_line_business_mailing_address_1,provider_business_mailing_address_city_name_1,provider_business_mailing_address_state_name_1,provider_business_mailing_address_postal_code_1,facility_consolidation_1,taxonomy_switch_1,taxonomy_code_1,code_1,grouping_1,classification_1,specialization_1,definition_1,notes_1,display_name_1,section_1,to_npi,entity_type_code_2,provider_organization_name_(legal_business_name)_2,provider_last_name_(legal_name)_2,provider_first_name_2,provider_middle_name_2,provider_name_prefix_text_2,provider_name_suffix_text_2,provider_credential_text_2,provider_other_organization_name_2,provider_first_line_business_mailing_address_2,provider_second_line_business_mailing_address_2,provider_business_mailing_address_city_name_2,provider_business_mailing_address_state_name_2,provider_business_mailing_address_postal_code_2,facility_consolidation_2,taxonomy_switch_2,taxonomy_code_2,code_2,grouping_2,classification_2,specialization_2,definition_2,notes_2,display_name_2,section_2,full_addy_1,full_addy_2,from_npi_pt_count,from_npi_transaction_count,to_npi_pt_count,to_npi_transaction_count,from_facility_pt_count,from_facility_transaction_count,to_facility_pt_count,to_facility_transaction_count
136442,1013991850,58,59,46.305,82.347,1.0,None,NG,CHRISTOPHER,C,DR.,None,MD,None,210 25TH AVE N STE 1204,None,NASHVILLE,TN,372031620.0,-,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,1003230236,Individual,2.0,"APP OF TENNESSEE ED, PLLC",None,None,None,None,None,None,None,PO BOX 31957,None,CLARKSVILLE,TN,370400033.0,-,Y,207P00000X,207P00000X,Allopathic & Osteopathic Physicians,Emergency Medicine,None,An emergency physician focuses on the immediat...,"Source: American Board of Medical Specialties,...",Emergency Medicine Physician,Individual,"210 25TH AVE N STE 1204, NASHVILLE","PO BOX 31957, CLARKSVILLE",15101,15710,36666932,52211903,991514,1062764,209804,243135
136443,1013991850,496,499,57.595,65.541,1.0,None,NG,CHRISTOPHER,C,DR.,None,MD,None,210 25TH AVE N STE 1204,None,NASHVILLE,TN,372031620.0,-,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,1003863580,Individual,2.0,"ASSOCIATED PATHOLOGISTS, LLC",None,None,None,None,None,None,ASSOCIATED PATHOLOGISTS LLC DBA PATHGROUP,5301 VIRGINIA WAY,SUITE 300,BRENTWOOD,TN,370277542.0,-,Y,207ZP0102X,207ZP0102X,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,A pathologist deals with the causes and nature...,"Source: American Board of Medical Specialties,...",Anatomic Pathology & Clinical Pathology Physician,Individual,"210 25TH AVE N STE 1204, NASHVILLE","5301 VIRGINIA WAY, BRENTWOOD",15101,15710,36666932,52211903,991514,1062764,1894619,2431116
136444,1013991850,642,665,22.771,49.368,1.0,None,NG,CHRISTOPHER,C,DR.,None,MD,None,210 25TH AVE N STE 1204,None,NASHVILLE,TN,372031620.0,-,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,1023055126,Individual,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,None,None,None,None,TRISTAR CENTENNIAL MEDICAL CENTER,2300 PATTERSON ST,None,NASHVILLE,TN,372031538.0,-,Y,282N00000X,282N00000X,Hospitals,Gener

In [51]:
# all of the times Vanderbilt was referred TO
full_2_Vandy= full_2[full_2['facility_consolidation_2']=='Vanderbilt']

In [58]:
# (4)group by from_npi summing the patient_count and then sorting the values for Vanderbilt referals
vandy_pt_count_by_from_npi= full_2_Vandy.groupby(['from_npi'])['patient_count'].sum().reset_index()
vandy_pt_count_by_from_npi. sort_values('patient_count', ascending=False)
#these are the npi's with the 5 most and least pt count referred to Vanderbilt

,from_npi,patient_count
391,1093753303,13363
1441,1316983695,8847
3356,1740377845,8838
166,1043302466,8219
955,1215966726,7222
...,...,...
1931,1427045830,17
616,1144228107,17
2533,1558515437,14
1543,1346312477,13


In [59]:
# (4)group by from_npi summing the transaction_count and then sorting the values for Vanderbilt referals
vandy_transaction_count_by_from_npi= full_2_Vandy.groupby(['from_npi'])['transaction_count'].sum().reset_index()
vandy_transaction_count_by_from_npi. sort_values('transaction_count', ascending=False)
#these are the npi's with the 5 most and least transaction count referred to Vanderbilt

,from_npi,transaction_count
391,1093753303,17194
741,1174517593,13187
1441,1316983695,11324
3356,1740377845,10680
171,1043394166,9887
...,...,...
2452,1548249188,50
3666,1811045966,50
2134,1467704643,50
1427,1316927999,50


In [60]:
# (4b)group by full address summing the patient_count and then sorting the values for Vanderbilt referals. can use this to find 
# out the facilities by address
vandy_pt_count_by_full_addy_1= full_2_Vandy.groupby(['full_addy_1'])['patient_count'].sum().reset_index()
vandy_pt_count_by_full_addy_1. sort_values('patient_count', ascending=False)
#these are the facilities's with the 5 most and least pt count referred to Vanderbilt

,full_addy_1,patient_count
812,"3601 TVC, NASHVILLE",298918
1120,"719 THOMPSON LN STE 30330, NASHVILLE",185903
810,"3601 THE VANDERBILT CLINIC, NASHVILLE",72718
694,"3024 BUSINESS PARK CIR, GOODLETTSVILLE",56760
1122,"719 THOMPSON LN, NASHVILLE",45120
...,...,...
776,"342 FREY ST, ASHLAND CITY",21
189,"1233 17TH AVE S, NASHVILLE",20
576,"2300 21ST AVE S, NASHVILLE",17
771,"3401 W END AVE, NASHVILLE",14


In [61]:
# (4b)group by full_addy_1 summing the transaction_count and then sorting the values for Vanderbilt referals. can use this to find 
# out the facilities by address
vandy_transaction_count_by_full_addy_1= full_2_Vandy.groupby(['full_addy_1'])['transaction_count'].sum().reset_index()
vandy_transaction_count_by_full_addy_1. sort_values('transaction_count', ascending=False)
#these are the facilities's with the 5 most and least transaction count referred to Vanderbilt

,full_addy_1,transaction_count
812,"3601 TVC, NASHVILLE",455501
1120,"719 THOMPSON LN STE 30330, NASHVILLE",294053
810,"3601 THE VANDERBILT CLINIC, NASHVILLE",120540
694,"3024 BUSINESS PARK CIR, GOODLETTSVILLE",61875
1122,"719 THOMPSON LN, NASHVILLE",60829
...,...,...
690,"301 W MAIN ST, WAVERLY",50
584,"233 JACKSON MEADOWS DR, HERMITAGE",50
189,"1233 17TH AVE S, NASHVILLE",50
633,"270 E COURT AVE, SELMER",50


In [59]:
# (4)group by from_npi summing the transaction_count and then sorting the values for Vanderbilt referals
vandy_transaction_count_by_from_npi= full_2_Vandy.groupby(['from_npi'])['transaction_count'].sum().reset_index()
vandy_transaction_count_by_from_npi. sort_values('transaction_count', ascending=False)
#these are the npi's with the 5 most and least transaction count referred to Vanderbilt

,from_npi,transaction_count
391,1093753303,17194
741,1174517593,13187
1441,1316983695,11324
3356,1740377845,10680
171,1043394166,9887
...,...,...
2452,1548249188,50
3666,1811045966,50
2134,1467704643,50
1427,1316927999,50


In [65]:
# (4b)group by display_name_1 summing the patient_count and then sorting the values for Vanderbilt referals.
vandy_pt_count_by_full_specialization_1= full_2_Vandy.groupby(['display_name_1'])['patient_count'].sum().reset_index()
vandy_pt_count_by_full_specialization_1. sort_values('patient_count', ascending=False)
#these are all the specializations referred to Vanderbilt

,display_name_1,patient_count
41,Diagnostic Radiology Physician,273518
21,Cardiovascular Disease Physician,155715
72,Internal Medicine Physician,94439
46,Family Medicine Physician,49751
43,Emergency Medicine Physician,48971
47,Family Nurse Practitioner,48033
94,Ophthalmology Physician,44886
14,Anatomic Pathology & Clinical Pathology Physician,44488
16,Anesthesiology Physician,26213
52,Gastroenterology Physician,24820


In [66]:
# (4b)group by display_name_1 summing the transaction_count and then sorting the values for Vanderbilt referals.
vandy_transaction_count_by_specialization_1= full_2_Vandy.groupby(['display_name_1'])['transaction_count'].sum().reset_index()
vandy_transaction_count_by_specialization_1. sort_values('transaction_count', ascending=False)
#these are all the specializations referred to Vanderbilt

,display_name_1,transaction_count
41,Diagnostic Radiology Physician,297039
21,Cardiovascular Disease Physician,216455
72,Internal Medicine Physician,161372
46,Family Medicine Physician,76037
94,Ophthalmology Physician,67601
47,Family Nurse Practitioner,65554
63,Hematology & Oncology Physician,60565
43,Emergency Medicine Physician,54441
14,Anatomic Pathology & Clinical Pathology Physician,50340
79,Nephrology Physician,49274
